# Application de methodes de traitement automatique d'image à un fonds photographique d'archives historique

## Du pixel aux images - 32M7138

*Printemps 2025 - Université de Genève*

*Raphaël Rollinet (raphael.rollinet@unine.ch)*

## Introduction

Ce Markdown effectué dans le cadre du cours "Du pixel aux images : introduction au traitement des images 2D" traite de méthodes utilisant de l'intelligence artificiel que cela soit avec des modèles de deep learning ou un LLM multimodale appliqué au traitement de l'image, plus précisément de la détection d'objet sur un fonds d'images anciennes. Mon travail vise à tester différente méthode et à analyser les résultats optenu afin d'en faire la critique du point de vue d'un archiviste.

### Méthode de détection d'objet

La méthode utilisée dans le cadre de ce projet sont un mélanges de méthodes de deep learning pour le traitement d'image. Pour être exactes les méthodes fonctionnent en mode inférence uniquement, et exploitent des poids figés issus de l’entraînement supervisé de grands corpus d’images. Leurs spécifications sont les suivantes :

#### Détection d’objets par SSD MobileNet v2 (TensorFlow Hub) :
Le modèle utilise une architecture Single Shot MultiBox Detector (SSD) avec un backbone MobileNetV2 optimisé pour les environnements à faibles ressources. SSD génère un ensemble de boîtes candidates (anchors) à différentes échelles et positions dans l’image, puis prédit pour chacune une probabilité de présence d’objet et une régression de boîte englobante. Le modèle a été préalablement entraîné sur COCO dataset, ce qui permet une détection multi-classes directement en inférence.

#### Classification d’image par ViT (Vision Transformer) :
Le modèle ViT Base (patch16-224) repose sur une adaptation des transformeurs à la vision par un découpage de l’image en patchs de 16x16, qui sont ensuite linéarisés et projetés en embeddings. Ces vecteurs sont enrichis par des encodages positionnels et traités par plusieurs couches transformeurs (self-attention). Le modèle prédit une seule classe globale pour l’image en sortie du token [CLS]. Il est pré-entraîné sur ImageNet-21k puis fine-tuné sur ImageNet-1k.

### Fonds d'images de la commune de Milvignes

Les modèles de détections et classifications des images sera appliqués à un fonds d'images provenant de l'inventaire des archives de la commune de Milvignes, membre du service intercommunal d'archives de Neuchâtel, la plateforme de diffusion des inventaires est gérée par docuteam SA, entreprise de gestion d'archives qui m'a employé de 2016 à 2021. Les images numérisées par mes soins en format de conservation (TIFF) et archivées électroniquement dans un repository de préservation numérique et également disponible  disponible en téléchargement librement dans un format dégradé de diffusion (JPEG).

![inventaire.png](attachment:inventaire.png)

Les images sont disponibles ici : https://milvignes.docuteam.cloud/fr/units/1-archives-communales-de-milvignes-1326-2013/gallery



## Problématique

Ayant longuement travaillé dans le domaine des archives, la thématique d'appliquer les méthodes vues dans le cadre de ce cours sur un fonds d'archives est un sujet intéressant dans son potentiel d'application sur un fonds d'archives d'images. Dans les institutions patrimoniales, les images sont un support courant, ma problématique sera donc appliquée à un fonds ancien d'images numérisées de la commune de Milvignes (NE). Mon questionnement porte sur l'application de cette méthode utilisant des modèles de deep learning entrainé sur des images relativement récentes, mais ici appliqué à un fonds d'images anciennes (~1900).

### Question de recherche

- "Quel est le potentiel de la détection d'objet dans le traitement, particulièrement pour la description de fonds d'images en archives ?"

## Méthodologie

Le code utilisé est adapté du code vu en cours, il vise à effectuer une détection d'objet sur des images, tout en répondant à des contraintes métiers qui sont par exemple un besoin d'automatiser le traitement à partir d'un workflow. Mais également en termes de rendu, un archiviste aura besoin d'extraire les détections sous format CSV afin qu'elles soient intégrées aux notices d'un système d'information archivistique par exemple sous la forme d'une indexation. Le code permet à n'importe quel archiviste avec des notions informatiques minimales de l'exécuter.

In [ ]:
#Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Installation des librairies (si nécessaire)
!pip install opencv-python
!pip install tensorflow_hub
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 113.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
# Importation des librairies Python
import os
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import pandas as pd
from PIL import Image
from transformers import ViTImageProcessor, ViTForImageClassification

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
# Charger les modèles (détection + classification)

# Modèle de détection SSD MobileNet V2
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1")

# Modèle ViT pour classification d'image
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [ ]:
# Liste des noms de classes (80 classes COCO)
classe = np.array([
    'background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
])
classes = np.concatenate([classe, np.repeat('none', 100)])


In [ ]:
#  À adapter : Mettre le dossier Images dans Google Drive
folder_path = "/content/drive/MyDrive/Colab_Notebooks/images"  # Adapter le chemin selon l'utilisateur

In [ ]:
# Liste pour stocker les résultats
results = []

In [ ]:
# Création d'une fonction de detection et classification pour le traitement de l'image
## Traite une image avec SSD MobileNet (détection) et ViT (classification).
## Sauvegarde l'image annotée et stocke les résultats dans la liste globale 'results'.

def process_image_with_detection_and_classification(image_path):

    # Lire l'image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Erreur : impossible de lire {image_path}")
        return

    # Convertir en RGB pour affichage et modèles
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    input_tensor = tf.convert_to_tensor(image_rgb)[tf.newaxis, ...]

    # Détection d'objets
    detections = detector(input_tensor)
    boxes = detections['detection_boxes'][0].numpy()
    scores = detections['detection_scores'][0].numpy()
    detected_classes = detections['detection_classes'][0].numpy().astype(np.uint8)

    # Préparer les annotations et objets détectés
    object_labels = []
    height, width, _ = image.shape
    for i in range(len(scores)):
        if scores[i] < 0.3:
            continue
        label = classes[detected_classes[i]]
        object_labels.append(label)

        # Boîte englobante
        box = boxes[i] * np.array([height, width, height, width])
        box = box.astype(np.int32)
        cv2.rectangle(image_rgb, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)
        cv2.putText(image_rgb, label, (box[1], box[0]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Classification avec ViT
    pil_image = Image.fromarray(image_rgb)
    inputs = processor(images=pil_image, return_tensors="pt")
    outputs = vit_model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    predicted_label = vit_model.config.id2label[predicted_class_idx]

    # Sauvegarde de l’image annotée dans /Resultat/annotated
    result_folder = os.path.join(folder_path, "Resultat")
    annotated_folder = os.path.join(result_folder, "annotated")
    os.makedirs(annotated_folder, exist_ok=True)

    save_path = os.path.join(annotated_folder, os.path.basename(image_path))
    cv2.imwrite(save_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))

    # Affichage pour visualisation
    plt.figure(figsize=(8, 6))
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title(f"{os.path.basename(image_path)}\n→ ViT : {predicted_label}")
    plt.show()

    # Stockage des résultats pour export CSV
    results.append({
        "image": os.path.basename(image_path),
        "detected_objects": ", ".join(object_labels),
        "vit_prediction": predicted_label
    })



In [ ]:
# Appliquer à toutes les images présentes dans le dossiers "images"
supported_formats = ('.jpg', '.jpeg', '.png', '.bmp')
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(supported_formats)]

print(f"{len(image_files)} images trouvées.")

for image_path in image_files:
    process_image_with_detection_and_classification(image_path)



In [ ]:
# Créer un sous-dossier "Résultat" dans le dossier Drive d'origine
result_folder = os.path.join(folder_path, "Resultat")
os.makedirs(result_folder, exist_ok=True)

# Définir le chemin de sortie du CSV dans ce dossier
csv_output_path = os.path.join(result_folder, "resultats_detection_classification.csv")

# Export du DataFrame en CSV
df_results = pd.DataFrame(results)
df_results.to_csv(csv_output_path, index=False, encoding='utf-8')

print(f"CSV enregistré ici : {csv_output_path}")


## Résultat

Les résultats appliqués à un fonds d'images anciennes sont mitigés. Le code a parfaitement fonctionné et le traitement de détection et classification a parfaitement été appliqué sur le fonds d'images. Cependant les images utilisées mettent en évidence certaines limitations de cette méthode.

La première provient probablement des images elles-mêmes, il s'agit de JPEG utilisé pour la diffusion sur le web avec une faible résolution. Les modèles se basant sur les pixels de l'image pour détecter les objets, cela influence le résultat final.

Deuxièmement les modèles utilisés ont été entrainés sur des images récentes, cela peut-être observé notamment avec les classes "COCO". Le modèle fonctionnant par inférence peu importe l'image, il y a un taux d'erreur plus important sur des images anciennes. Cela peut être partiellement palié en enlevant des classes. Neanmoins cela réste un biais inhérant à ces modèles.

# Traitement par LLM

In [ ]:
# -- Étape 1 : Installer et importer les bibliothèques nécessaires --
!pip install openai pillow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 26.0 MB/s eta 0:00:00


In [ ]:
import base64
import os
import time
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from PIL import Image, UnidentifiedImageError

In [ ]:
# --- Étape 2 : Configuration OpenRouter ---
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"  # Remplace par ta clé OpenRouter
)

In [ ]:
# --- Étape 3 : Fonctions utilitaires ---
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

def is_image_valid(image_path):
    try:
        Image.open(image_path).verify()
        return True
    except (UnidentifiedImageError, IOError):
        return False

def query_llama_vision(image_path):
    base64_image = encode_image_to_base64(image_path)

    response = client.chat.completions.create(
        model="meta-llama/llama-3.2-11b-vision-instruct:free",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                    },
                    {
                        "type": "text",
                        "text": "Provide a short descriptive caption for this archival photograph in English, focusing on historical or visual context."
                    }
                ]
            }
        ],
        max_tokens=500
    )
    return response.choices[0].message.content.strip()

def query_llama_vision_with_retry(image_path, retries=3, delay=2):
    for attempt in range(retries):
        try:
            if not is_image_valid(image_path):
                return "[ERROR] Unreadable or corrupted image"
            return query_llama_vision(image_path)
        except Exception as e:
            if attempt < retries - 1:
                print(f"⚠️ Attempt {attempt+1} failed: {e}. Retrying in {delay} sec...")
                time.sleep(delay)
            else:
                return f"[ERROR] {str(e)}"

In [ ]:
# --- Étape 4 : Paramètres de chemin ---
folder_path = "/content/drive/MyDrive/Colab_Notebooks/images"
result_folder = "/content/drive/MyDrive/Colab_Notebooks/images/Resultat"
os.makedirs(result_folder, exist_ok=True)

In [ ]:
# --- Étape 5 : Liste des fichiers image ---
image_filenames = sorted([
    f for f in os.listdir(folder_path)
    if f.lower().endswith(('.jpg', '.jpeg', '.png'))
])

In [ ]:
# --- Étape 6 : Génération des descriptions avec rendu ---
results_llm = []
for filename in tqdm(image_filenames, desc="🔎 Interrogation de LLaMA 3 Vision"):
    image_path = os.path.join(folder_path, filename)
    llm_description = query_llama_vision_with_retry(image_path)
    results_llm.append({
        "filename": filename,
        "llm_description": llm_description
    })
    time.sleep(2)  # Respect des limites d'OpenRouter


🔎 Interrogation de LLaMA 3 Vision:   6%|▌         | 1/18 [00:05<01:29,  5.27s/it]

⚠️ Attempt 1 failed: 'NoneType' object is not subscriptable. Retrying in 2 sec...


🔎 Interrogation de LLaMA 3 Vision:  11%|█         | 2/18 [00:12<01:44,  6.54s/it]

⚠️ Attempt 1 failed: 'NoneType' object is not subscriptable. Retrying in 2 sec...


🔎 Interrogation de LLaMA 3 Vision:  17%|█▋        | 3/18 [00:20<01:49,  7.27s/it]

...

🔎 Interrogation de LLaMA 3 Vision:  67%|██████▋   | 12/18 [01:27<00:42,  7.08s/it]

⚠️ Attempt 1 failed: 'NoneType' object is not subscriptable. Retrying in 2 sec...


🔎 Interrogation de LLaMA 3 Vision:  78%|███████▊  | 14/18 [01:40<00:26,  6.66s/it]

⚠️ Attempt 1 failed: 'NoneType' object is not subscriptable. Retrying in 2 sec...


🔎 Interrogation de LLaMA 3 Vision: 100%|██████████| 18/18 [02:03<00:00,  6.85s/it]


In [ ]:
# --- Étape 7 : Sauvegarde dans un CSV ---
csv_output_path = os.path.join(result_folder, "résultats_llm_llama32.csv")
pd.DataFrame(results_llm).to_csv(csv_output_path, index=False, encoding="utf-8")
print(f"✅ Résultats enregistrés dans : {csv_output_path}")

✅ Résultats enregistrés dans : /content/drive/MyDrive/Colab_Notebooks/images/Resultat/résultats_llm_llama32.csv


## Mise en perspectives et conclusion

En conclusion, le fonctionnement du code permettant un traitment par lot, rend la méthode intéressante sur des fonds d'images en archives. De plus le rendu en CSV permet des imports, permettant d'utiliser ces classifications comme methode d'indexation automatique.

## Bibliographie